## 5-2 교차 검증과 그리드 서치

## 검증 세트

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split

# 훈련 세트(train_input, train_target)와 테스트 세트(test_input, test_target)로 나눔.
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

# 훈련 세트를 다시 훈련세트(sub_target, sub_target)와 검증세트(val_input, val_target)로 나눔.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

# 훈련 세트와 검증 세트의 크기
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# 훈련 세트와 검증 세트를 사용해 모델 생성 후 평가
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 훈련 세트에 과대적합 되어 있음.
# 매개 변수 바꾸어 더 좋은 모델 찾아야 함.

0.9971133028626413
0.864423076923077


## 교차 검증

In [ ]:
from sklearn.model_selection import cross_validate

# cross_validate(평가할 모델 객체, 훈련 세트 전체) : 교차 검증 함수, 기본적으로 5-폴드 교차 검증 수행.
scores = cross_validate(dt, train_input, train_target)
print(scores)

# fit_time : 모델 훈련하는 시간
# score_time : 모델 검증하는 시간
# test_score : 각 모델 평가 점수.

{'fit_time': array([0.0101583 , 0.01259565, 0.01045156, 0.01030135, 0.00973535]), 'score_time': array([0.00091505, 0.00077534, 0.00092268, 0.00088334, 0.00085545]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

# 교차 검증의 최종점수 : test_score에 담긴 5개 점수 평균화하여 얻음.
print(np.mean(scores['test_score']))

from sklearn.model_selection import StratifiedKFold

# 앞선 단계에서는 train_test_split()를 이용하여 전체 데이터를 섞은 후 훈련 세트를 준비했기에 따로 섞을 필요가 없었음.
# 교차 검증시 훈련 세트 섞기 위해 분할기(splitter) 지정해야 함.
# cross_validate() : 회귀 모델일 경우 KFold 분할기 사용, 분류 모델일 경우 타깃 클래스 골고루 나눌 수 있는 StratifiedKFold 사용.
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

# 훈련 세트 섞은 후 10-폴드 교차 검증 수행하는 법
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.855300214703487
0.8574181117533719


## 하이퍼파라미터 튜닝

In [ ]:
# min_impurity_decrease 매개변수의 최적값 찾아보기
from sklearn.model_selection import GridSearchCV

# 1. 탐색할 매개변수 지정.

# 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦.
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# 2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾음. 이 조합은 그리드 서치 객체에 저장.

# GridSearchCV 클래스에 탐색할 대상 모델, params 변수 전달하여 그리드 서치 객체 생성.
# GridSearchCV() : cv 기본값 = 5, min_impurity_decrease 값마다 5-폴드 교차검증 수행, 5 * 5 = 25 개의 모델 훈련.
# n_jobs : 병렬 실행에 사용할 CPU 코어 수 지정, 기본값 = 1, -1로 지정시 시스템의 모든 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# fit() 호출 시 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행.
gs.fit(train_input, train_target)

# 사이킷런의 그리드 서치는 훈련이 끝나면 자동으로 25개의 모델 중 검증 점수가 가장 높은 모델의 매개변수 조합으로 다시 모델 훈련
# 이 모델은 gs.best_estimator_에 저장되어 있음.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

# gs.best_params_ : 그리드 서치로 찾은 최적의 매개 변수 저장.
print(gs.best_params_)

# 3. 그리드 서치는 매개 변수에서 전체 훈련 세트를 사용해 최종 모델 훈련. 이 모델도 그리드 서치 객체에 저장됨.

# cv_results_['mean_test_score'] : 각 매개변수에서 수행한 교차 검증의 평균 점수 저장.
print(gs.cv_results_['mean_test_score'])

# argmax() : 가장 큰 값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])

# params키에 저장된 매개변수, 즉 최상의 검증 점수를 만든 매개변수 조합
# gs.best_params_ 와 동일
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [ ]:
# 더 복잡한 매개변수 조합 탐색
# min_impurity_decrease : 노드 분할을 위한 불순도 감소 최소량,
# max_depth : 트리 깊이 제한
# min_samples_split : 노드를 나누기 위한 최소 샘플 수
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # 9개
          'max_depth': range(5, 20, 1), # 15개
          'min_samples_split': range(2, 100, 10) # 10개
          }
# 이 매개변수로 수행할 교차 검증 횟수 : 9 * 15 * 10 = 1350
# 기본 5-폴드 교차검증 수행하므로 만들어지는 모델 수 : 1350 * 5 = 6750

# 그리드 서치 실행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

# 최상의 매개변수 조합 확인
print(gs.best_params_)

# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


## 랜덤 서치

In [ ]:
# uniform, randint : 주어진 범위에서 고르게 값을 뽑음 = 균등 분포에서 샘플링함.
# randint : 정수값 뽑음 / uniform : 실숫값 뽑음.
from scipy.stats import uniform, randint

rgen = randint(0, 10)
# rvs(표본 생성시 생성될 표본의 크기)
rgen.rvs(10)

array([0, 2, 5, 8, 4, 5, 2, 3, 8, 4])

In [ ]:
# unique() : 주어진 numpy배열에서 모든 고유 값들을 검색하고 이러한 고유값들을 정렬
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106, 106, 105, 113,  90, 109,  79,  91, 105,  96]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.23646165, 0.4590276 , 0.50746045, 0.05409416, 0.90979408,
       0.04435646, 0.48869753, 0.08397129, 0.11242201, 0.87376509])

In [ ]:

# 탐색할 매개변수 범위
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25), # 리프 노드가 되기 위한 최소 샘플의 개수 : 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음.
          }

from sklearn.model_selection import RandomizedSearchCV

# n_iter : 샘플링 횟수
# n_iter번 샘플링하여 교차 검증 수행 후 최적의 매개변수 조합을 찾음.
# 그리드 서치보다 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색 가능.
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

# 최적의 매개변수 조합 출력
print(gs.best_params_)

# 최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

# 최적 모델은 이미 전체 훈련 세트로 훈련되어 best_estimator_ 속성에 저장되어 있음.
# 이 모델을 최종 모델로 결정한 후 테스트 세트 성능 확인.
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
